In [68]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import requests
import time
from ultralytics import YOLO
# from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor



In [32]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [69]:
# sam_checkpoint = "/content/drive/MyDrive/sam_vit_h_4b8939.pth"
sam_checkpoint = "/content/drive/MyDrive/mobile_sam.pt"
# model_type = "vit_h"
model_type = "vit_t"
device="cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

In [34]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [35]:
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

In [36]:
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


In [37]:
def normalize_points(contour_points, image_width, image_height):
    # Normalize the points
    normalized_points = contour_points.astype(float) / np.array([image_width, image_height])
    return normalized_points

In [38]:
def convert_contour_to_yolov8(contour_points, class_index):
    # Flatten the contour points to a 1D array
    flattened_points = contour_points.reshape(-1, 2)

    # Create YOLOv8 label string
    label_string = f"{class_index}"
    for point in flattened_points:
        label_string += f" {point[0]} {point[1]}"
    return label_string

In [39]:
def download_image_from_drive(url, destination):
    response = requests.get(url)
    response.raise_for_status()
    with open(destination, 'wb') as file:
        file.write(response.content)
    print("Image downloaded successfully.")



In [40]:
image_url = "https://drive.google.com/uc?id=1RjaSIfNz11fcRvWgL1ER4ZWttxIZ8zzY"
destination_path = "/content/drive/MyDrive/data/images/images.jpg"

In [41]:
def extract_roi(image_path,coord_list):
    image=cv2.imread(image_path)
    print("Coord List:", coord_list)
    x1, y1, x2, y2 = coord_list
    roi = image[y1:y2, x1:x2]
    # roi = image[y2:y1, x1:x2]
    return roi

In [42]:
def load_model(model_path):
    model=YOLO(model=model_path)
    return model

In [43]:
def extract_labels(roi,model):

    img_dir="/content/drive/MyDrive/data/images/"
    img_list = os.listdir(img_dir)
    img_list.sort()
    image_path=img_dir + img_list[0]
    print(image_path)
    print(img_list[0])

    results=model.predict(source=roi,conf=0.5,save=True,show_labels=False)
    contours=[]
    # boxes=result.boxes
    for result in results:
        boxes = result.boxes

    i = 0
    label_string_list =[]
    while i < len(boxes.xyxy.tolist()):
        bbox = boxes.xyxy.tolist()[i]
        class_id = boxes.cls.tolist()[i]
        # print(bbox)

        # image = cv2.cvtColor(cv2.imread(roi), cv2.COLOR_BGR2RGB)
        predictor.set_image(roi)
        input_box = np.array(bbox)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box,
            multimask_output=False,
        )
        # print(masks)
        mask = masks[0]
        # print(mask)
        contour, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours.extend(contour)

        contour_points = contours[i]
        class_index = int(class_id)
        image_height, image_width = roi.shape[:2]

        normalized_points = normalize_points(contour_points, image_width, image_height)
        label_string = convert_contour_to_yolov8(normalized_points, class_index)
        label_string_list.append(label_string)
        i += 1
    return label_string_list
    # output_path =image_path.split(".")[0] + '.txt'
    # print(output_path)
    # with open(output_path, 'w') as file:
    #     for item in label_string_list:
    #         file.write(str(item) + '\n')


In [72]:
start_time=time.time()
model_path="/content/drive/MyDrive/best_detect.pt"
model=load_model(model_path)
bounding_box_list=[[1046, 298, 1152, 419],[720,217,862,320],[755,351,899,463],[689,85,832,184],[355,446,568,548],[583,403,754,492],[0,57,258,239],[545,254,715,328],[277,62,489,209],[888,184,1026,297],[15,278,290,436],[978,75,1112,165],[48,466,327,645],[920,314,1044,423],[1046,298,1152,419]]
images_path="/content/VID_20220718_161715_0009.jpg"
download_image_from_drive(image_url, destination_path)
for i in range(len(bounding_box_list)):
  roi=extract_roi(destination_path,bounding_box_list[i])
  label_string_list=extract_labels(roi,model)
  output_path =destination_path.split(".")[0] + f'{i}.txt'
  print(output_path)
  with open(output_path, 'w') as file:
      for item in label_string_list:
            file.write(str(item) + '\n')
  file.close()
  with open(output_path, 'r') as file:
      lines = file.readlines()
      num_ones = 0
      num_zeros = 0
      for line in lines:
        line_list = line.split()
        class_label = int(line_list[0])
        if class_label == 1:
            num_ones += 1
        elif class_label == 0:
            num_zeros += 1
      print("Number of emptys:", num_ones)
      print("Number of objects:", num_zeros)
end_time=time.time()
print("Total Time :",end_time-start_time)






Image downloaded successfully.
Coord List: [1046, 298, 1152, 419]
/content/drive/MyDrive/data/images/images.jpg
images.jpg



0: 640x576 1 object, 17.2ms
Speed: 2.5ms preprocess, 17.2ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs/detect/predict34

0: 480x640 3 objects, 22.1ms
Speed: 2.9ms preprocess, 22.1ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict34


/content/drive/MyDrive/data/images/images0.txt
Number of emptys: 0
Number of objects: 1
Coord List: [720, 217, 862, 320]
/content/drive/MyDrive/data/images/images.jpg
images.jpg



0: 512x640 3 objects, 13.6ms
Speed: 3.0ms preprocess, 13.6ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict34


/content/drive/MyDrive/data/images/images1.txt
Number of emptys: 0
Number of objects: 3
Coord List: [755, 351, 899, 463]
/content/drive/MyDrive/data/images/images.jpg
images.jpg


...

/content/drive/MyDrive/data/images/images11.txt
Number of emptys: 0
Number of objects: 9
Coord List: [48, 466, 327, 645]
/content/drive/MyDrive/data/images/images.jpg
images.jpg



0: 576x640 3 objects, 8.7ms
Speed: 3.6ms preprocess, 8.7ms inference, 1.5ms postprocess per image at shape (1, 3, 576, 640)
Results saved to runs/detect/predict34


/content/drive/MyDrive/data/images/images12.txt
Number of emptys: 6
Number of objects: 0
Coord List: [920, 314, 1044, 423]
/content/drive/MyDrive/data/images/images.jpg
images.jpg



0: 640x576 1 object, 8.4ms
Speed: 4.1ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs/detect/predict34


/content/drive/MyDrive/data/images/images13.txt
Number of emptys: 0
Number of objects: 3
Coord List: [1046, 298, 1152, 419]
/content/drive/MyDrive/data/images/images.jpg
images.jpg
/content/drive/MyDrive/data/images/images14.txt
Number of emptys: 0
Number of objects: 1
Total Time : 8.117621183395386


In [1]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.9/605.9 kB 3.8 MB/s eta 0:00:00


In [2]:
pip install segment_anything

In [64]:
pip install git+https://github.com/ChaoningZhang/MobileSAM.git

  Cloning https://github.com/ChaoningZhang/MobileSAM.git to /tmp/pip-req-build-wypfj2o4
  Running command git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/MobileSAM.git /tmp/pip-req-build-wypfj2o4
  Resolved https://github.com/ChaoningZhang/MobileSAM.git to commit 01ea8d0f5590082f0c1ceb0a3e2272593f20154b
  Preparing metadata (setup.py) ... done
  Created wheel for mobile-sam: filename=mobile_sam-1.0-py3-none-any.whl size=42433 sha256=c0311e3ea7b5828ae811d2636acfb557d0051e1cde82c495e874fb5d48b73fb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-8g6c6g37/wheels/43/b1/9d/1c1b33c31d4c54f0a502f9c48b655f87213ab01e55d09cf4ef
Successfully built mobile-sam


In [67]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
